In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pima-indians-diabetes-database/diabetes.csv


In [2]:
path = '/kaggle/input/pima-indians-diabetes-database/diabetes.csv'
df = pd.read_csv(path)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [4]:
x = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x = scaler.fit_transform(x)

In [6]:
x.shape

(768, 8)

In [7]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test  = train_test_split(x, y, test_size=0.2)

In [8]:
x_train.shape

(614, 8)

In [9]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout


In [10]:
# Basic initial model
model = Sequential()

model.add(Dense(32, activation='relu', input_dim=8))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
model.fit(x_train, y_train, batch_size=32, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
20/20 [==============================] - 1s 14ms/step - loss: 0.6410 - accuracy: 0.6726 - val_loss: 0.6121 - val_accuracy: 0.6818
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5821 - accuracy: 0.7345 - val_loss: 0.5765 - val_accuracy: 0.7208
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5463 - accuracy: 0.7524 - val_loss: 0.5547 - val_accuracy: 0.7403
Epoch 4/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5245 - accuracy: 0.7557 - val_loss: 0.5411 - val_accuracy: 0.7532
Epoch 5/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5095 - accuracy: 0.7590 - val_loss: 0.5302 - val_accuracy: 0.7532
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4984 - accuracy: 0.7622 - val_loss: 0.5220 - val_accuracy: 0.7532
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4897 - accuracy: 0.7671 - val_loss: 0.5153 - val_accuracy: 0.7597
Epoch 8/100


In [12]:
# Parameter that we want to optimize
# 1. Select appropriate optimizer
# 2. No of nodes in a layer
# 3. No of layers in the model
# 4. All in one model

In [13]:
import keras_tuner as kt

In [14]:
# Finding the best optimizer
def build_model(hp):
    
    model = Sequential()
    
    model.add(Dense(32, activation='relu', input_dim = 8))
    model.add(Dense(1, activation='sigmoid'))
    
    optimizer = hp.Choice('optimizer',values =  ['adam', 'sgd', 'rmsprop', 'adadelta'])
                          
    model.compile(optimizer=optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [15]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials = 5,
    overwrite = 'True'
)

In [16]:
tuner.search(x_train, y_train, epochs=5, validation_data = (x_test, y_test))

Trial 4 Complete [00h 00m 01s]
val_accuracy: 0.7207792401313782

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 06s


In [17]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [18]:
# selecting the best model
best_model = tuner.get_best_models(num_models=1)[0]

In [19]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                288       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [20]:
best_model.fit(x_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(x_test, y_test))

Epoch 7/100
20/20 [==============================] - 1s 13ms/step - loss: 0.4993 - accuracy: 0.7704 - val_loss: 0.5059 - val_accuracy: 0.7857
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4894 - accuracy: 0.7704 - val_loss: 0.4998 - val_accuracy: 0.7922
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4811 - accuracy: 0.7720 - val_loss: 0.4958 - val_accuracy: 0.7922
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4747 - accuracy: 0.7818 - val_loss: 0.4916 - val_accuracy: 0.7857
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4697 - accuracy: 0.7850 - val_loss: 0.4881 - val_accuracy: 0.7727
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4656 - accuracy: 0.7866 - val_loss: 0.4864 - val_accuracy: 0.7792
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4619 - accuracy: 0.7932 - val_loss: 0.4851 - val_accuracy: 0.7727
Epoch 14

In [21]:
# No of nodes in a layer
def build_model(hp):
    
    model = Sequential()
    
    units = hp.Int('units', min_value = 8, max_value = 128, step = 8)
    
    model.add(Dense(units = units, activation = 'relu', input_dim = 8))
    
    model.add(Dense(1, activation = 'sigmoid'))
    
    model.compile(
        optimizer = 'adam',
        loss = 'binary_crossentropy',
        metrics = ['accuracy']
    )
    
    return model

In [22]:
tuner = kt.RandomSearch(
    build_model,
    objective = 'val_accuracy',
    max_trials=5,
    directory='mydir',
    project_name = 'units',
    overwrite = 'True',
)

In [23]:
tuner.search(x_train, y_train,epochs = 5, validation_data = (x_test, y_test))

Trial 5 Complete [00h 00m 01s]
val_accuracy: 0.7597402334213257

Best val_accuracy So Far: 0.7662337422370911
Total elapsed time: 00h 00m 07s


In [24]:
tuner.get_best_hyperparameters()[0].values

{'units': 72}

In [25]:
best_model = tuner.get_best_models()[0]

In [26]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 72)                648       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 73        
Total params: 721
Trainable params: 721
Non-trainable params: 0
_________________________________________________________________


In [27]:
best_model.fit(x_train, y_train, epochs = 100, batch_size = 32, initial_epoch=6, validation_data=(x_test, y_test))

Epoch 7/100
20/20 [==============================] - 1s 12ms/step - loss: 0.4858 - accuracy: 0.7769 - val_loss: 0.5063 - val_accuracy: 0.7468
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4750 - accuracy: 0.7818 - val_loss: 0.4977 - val_accuracy: 0.7403
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4659 - accuracy: 0.7834 - val_loss: 0.4922 - val_accuracy: 0.7468
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4598 - accuracy: 0.7899 - val_loss: 0.4907 - val_accuracy: 0.7597
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4554 - accuracy: 0.7834 - val_loss: 0.4878 - val_accuracy: 0.7662
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4509 - accuracy: 0.7899 - val_loss: 0.4866 - val_accuracy: 0.7662
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4481 - accuracy: 0.7866 - val_loss: 0.4863 - val_accuracy: 0.7597
Epoch 14

In [28]:
def build_model(hp):
    
    model = Sequential()
    
    model.add(Dense(72, activation='relu', input_dim = 8))
    
    for i in range(hp.Int('num_layers', min_value = 1, max_value = 10)):
        
        model.add(Dense(72, activation='relu'))
    
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer= 'adam', loss = 'binary_crossentropy', metrics =['accuracy'])

    return model

In [29]:
tuner = kt.RandomSearch(
    build_model, 
    objective='val_accuracy', 
    max_trials = 3, 
    directory='mydir',
    project_name = 'layers',
    overwrite = 'True',)

In [30]:
tuner.search(x_train, y_train,epochs=5, validation_data = (x_test, y_test))

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.7662337422370911

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 06s


In [31]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3}

In [32]:
best_model = tuner.get_best_models()[0]

In [33]:
best_model.fit(x_train, y_train, initial_epoch=6, epochs=100, validation_data=(x_test, y_test))

Epoch 7/100
20/20 [==============================] - 1s 16ms/step - loss: 0.4665 - accuracy: 0.7834 - val_loss: 0.4898 - val_accuracy: 0.7532
Epoch 8/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4509 - accuracy: 0.7850 - val_loss: 0.4887 - val_accuracy: 0.7662
Epoch 9/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4324 - accuracy: 0.7948 - val_loss: 0.4909 - val_accuracy: 0.7662
Epoch 10/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4231 - accuracy: 0.7899 - val_loss: 0.4884 - val_accuracy: 0.7403
Epoch 11/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4214 - accuracy: 0.8013 - val_loss: 0.4978 - val_accuracy: 0.7532
Epoch 12/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4202 - accuracy: 0.7997 - val_loss: 0.4848 - val_accuracy: 0.7532
Epoch 13/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4078 - accuracy: 0.7980 - val_loss: 0.4998 - val_accuracy: 0.7468
Epoch 14

In [34]:
#
def build_model(hp):
    model = Sequential()
    counter = 0
    for i in range(hp.Int('num_layers', min_value = 1, max_value = 10)):
        if counter  == 0:
            model.add(
                Dense(
                    hp.Int('units' + str(i), min_value = 8, max_value = 128, step = 8),
                    activation = hp.Choice('activation' + str(i), values=['tanh', 'relu', 'sigmoid']),
                    input_dim = 8
                    )
                )
            model.add(Dropout(hp.Choice('dropout' + str(i), values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        else:
            model.add(
                Dense(
                    hp.Int('units' + str(i), min_value = 8, max_value = 128, step = 8),
                    activation = hp.Choice('activation' + str(i), values=['tanh', 'relu', 'sigmoid'])
                    )
                )
            model.add(Dropout(hp.Choice('dropout' + str(i), values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        counter += 1
    model.add(Dense(1, activation= 'sigmoid'))
    
    model.compile(optimizer= hp.Choice('optimizer' + str(i), values=['adam', 'rmsprop', 'sgd', 'nadam', 'adadelta']),
                  loss = 'binary_crossentropy',
                  metrics = ['accuracy']
                 )
    
    return model

In [35]:
tuner = kt.RandomSearch(build_model,
                       objective = 'val_accuracy',
                       max_trials = 10,
                       directory='mydir',
                       project_name = 'all_pram',
                       overwrite = 'True', 
                        )

In [36]:
%%time
tuner.search(x_train, y_train, epochs = 5, validation_data = (x_test, y_test))

Trial 10 Complete [00h 00m 01s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 23s
CPU times: user 23.7 s, sys: 1.11 s, total: 24.8 s
Wall time: 23.7 s


In [37]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2,
 'units0': 40,
 'activation0': 'tanh',
 'dropout0': 0.6,
 'optimizer0': 'adadelta',
 'units1': 88,
 'activation1': 'tanh',
 'dropout1': 0.4,
 'units2': 112,
 'activation2': 'sigmoid',
 'dropout2': 0.9,
 'units3': 96,
 'activation3': 'relu',
 'dropout3': 0.6,
 'units4': 96,
 'activation4': 'tanh',
 'dropout4': 0.2,
 'units5': 32,
 'activation5': 'relu',
 'dropout5': 0.7,
 'units6': 32,
 'activation6': 'sigmoid',
 'dropout6': 0.8,
 'units7': 24,
 'activation7': 'sigmoid',
 'dropout7': 0.3,
 'units8': 104,
 'activation8': 'tanh',
 'dropout8': 0.1,
 'optimizer8': 'adadelta',
 'optimizer7': 'rmsprop',
 'optimizer5': 'sgd',
 'optimizer4': 'rmsprop',
 'optimizer1': 'adam'}

In [38]:
best_model = tuner.get_best_models()[0]

In [39]:
best_model.fit(x_train, y_train, epochs = 100, initial_epoch=5, validation_data = (x_test, y_test))

Epoch 6/100
20/20 [==============================] - 1s 14ms/step - loss: 0.5184 - accuracy: 0.7182 - val_loss: 0.4837 - val_accuracy: 0.7727
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5251 - accuracy: 0.7296 - val_loss: 0.4863 - val_accuracy: 0.7792
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5085 - accuracy: 0.7541 - val_loss: 0.4892 - val_accuracy: 0.7857
Epoch 9/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5331 - accuracy: 0.7280 - val_loss: 0.4890 - val_accuracy: 0.7727
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5015 - accuracy: 0.7492 - val_loss: 0.4918 - val_accuracy: 0.7857
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4953 - accuracy: 0.7573 - val_loss: 0.4918 - val_accuracy: 0.7792
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4863 - accuracy: 0.7801 - val_loss: 0.4895 - val_accuracy: 0.7727
Epoch 13/

In [40]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 40)                360       
_________________________________________________________________
dropout (Dropout)            (None, 40)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 88)                3608      
_________________________________________________________________
dropout_1 (Dropout)          (None, 88)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 89        
Total params: 4,057
Trainable params: 4,057
Non-trainable params: 0
_________________________________________________________________
